# JDからの情報抽出

Building Effective Agents Cookbook: https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents  
Evaluator-Optimizer Workflow: https://github.com/anthropics/anthropic-cookbook/blob/main/patterns/agents/evaluator_optimizer.ipynb

In [16]:
from dotenv import load_dotenv
import os
import json
import re

import pandas as pd
import pdfplumber

from h2ogpte import H2OGPTE

In [2]:
jd_path = '../JDs/'
pdf_files = [file for file in os.listdir(jd_path) if file.endswith(".pdf")]
pdf_files

['Digital BCG - Data Scientist.pdf',
 'Robust Intelligence - Machine Learning Engineer.pdf',
 'Eli Lilly - Data Scientist, Global Report.pdf',
 'OpenAI - Principal Solutions Engineer – Japan.pdf',
 'Money Forward - AI Soultions Engineer.pdf',
 'AWS - Sr. Generative AI Specialist, Generative AI Innovation Center.pdf',
 'Senior Data Scientist @ Dataiku.pdf']

In [3]:
load_dotenv(dotenv_path='.env')
heogpte_key = os.getenv('h2oGpteKey_AgentWorkflow')
if heogpte_key: print('key is set.')

key is set.


In [4]:
client = H2OGPTE(
    address='https://h2ogpte.dev.h2o.ai',
    api_key=heogpte_key,   # Collection Key
)

chat_session_id = client.create_chat_session_on_default_collection()

client, chat_session_id

(<h2ogpte.h2ogpte.H2OGPTE at 0x10a892770>,
 '00d2d44e-d852-4400-abfc-27df2ba9d0c7')

In [5]:
# 利用可能なLLM
#client.get_llm_names()

In [6]:
def llm_call(prompt: str, system_prompt: str = "あなたは優秀なAIアシスタントです。指示に的確に従います。", model='gpt-4o-mini') -> str:
    with client.connect(chat_session_id) as session:
        reply = session.query(
            message=prompt,
            #system_prompt=system_prompt,
            llm=model,
            rag_config = {"rag_type": "llm_only"},
            timeout=60,
        )
        return reply.content

def extract_xml(text: str, tag: str) -> str:
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""

In [7]:
def generate(prompt: str, task: str, context: str = "") -> tuple[str, str]:
    """Generate and improve a solution based on feedback."""
    full_prompt = f"{prompt}\n{context}\nTask: {task}" if context else f"{prompt}\nTask: {task}"
    #print('generate full_prompt >>>>>>>>>>>>>>>>>', full_prompt, '<<<<<<<<<<<<<<<<<<< generate full_prompt')
    response = llm_call(full_prompt)
    thoughts = extract_xml(response, "thoughts")
    result = extract_xml(response, "response")
    
    print("\n=== GENERATION START ===")
    print(f"Thoughts:\n{thoughts}\n")
    print(f"Generated:\n{result}")
    print("=== GENERATION END ===\n")
    
    return thoughts, result

def evaluate(prompt: str, content: str, task: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nOriginal task: {task}\nContent to evaluate: {content}"
    #print('evaluate full_prompt >>>>>>>>>>>>>>>>>', full_prompt, '<<<<<<<<<<<<<<<<<<< evaluate full_prompt')
    response = llm_call(full_prompt)
    evaluation = extract_xml(response, "evaluation")
    feedback = extract_xml(response, "feedback")
    
    print("=== EVALUATION START ===")
    print(f"Status: {evaluation}")
    print(f"Feedback: {feedback}")
    print("=== EVALUATION END ===\n")
    
    return evaluation, feedback

def loop(task: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    thoughts, result = generate(generator_prompt, task)
    memory.append(result)
    chain_of_thought.append({"thoughts": thoughts, "result": result})
    
    while True:
        evaluation, feedback = evaluate(evaluator_prompt, result, task)
        if evaluation == "PASS":
            return result, chain_of_thought
            
        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])
        
        thoughts, result = generate(generator_prompt, task, context)
        memory.append(result)
        chain_of_thought.append({"thoughts": thoughts, "result": result})

In [9]:
evaluator_prompt = """
与えられたテキストを以下2つの基準で判定してください。
1. 「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つ全ての項目が抽出されているか。なお「JDには記載なし」と出力されているものは正確に抽出されていると判断して問題ないです。
2.  抽出された情報はJSON形式となっているか。

タスクの解決を試みることなく、評価のみを行ってください。
すべての基準が満たされ、改善のための提案がない場合は、"PASS"のみを出力してください。
次の形式で簡潔に評価を出力してください。

<evaluation>PASS, NEEDS_IMPROVEMENT, or FAIL</evaluation>
<feedback>
改善が必要な理由。
</feedback>
"""

generator_prompt = """
目標は、<user input>に基づいてタスクを完了することです。
フィードバックを受け取った場合は、それを考慮して回答を改善する必要があります。

次の形式で簡潔に回答を出力してください。

<thoughts>
タスクとフィードバックに対する理解、および改善の計画。
</thoughts>

<response>
{
    "会社名":"回答",
    "ポジション名:"回答",
    "必要な学歴":"回答",
    "必要な経験年数":"回答",
    "必要な言語スキル":"回答",
    "必要なテクニカルスキル":"回答"
}
</response>
"""

In [10]:
reply_results = []

for pdf in pdf_files:
    print('START >>>>>>>>>> ', pdf)
    file_path = jd_path+pdf

    with pdfplumber.open(file_path) as pdf:
        text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    #print(text)

    task = """
    以下のJob Descriptionから、「会社名」「ポジション名」「必要な学歴」「必要な経験年数」「必要な言語スキル」「必要なテクニカルスキル」の6つを抽出してください。
    Job Description内に情報がない場合は、'JDには記載なし'と出力してください。
    
    # Job Description
    {}
    """.format(text)
    #print(task)

    reply = loop(task, evaluator_prompt, generator_prompt)
    reply_results.append(reply)

len(reply_results)

START >>>>>>>>>>  Digital BCG - Data Scientist.pdf

=== GENERATION START ===
Thoughts:

Job Descriptionから必要な情報を抽出するタスクを理解しました。フィードバックを受けた場合には、情報の正確性や明確さを向上させるために、より詳細な分析を行う計画です。特に、必要な学歴や経験年数、スキルに関する情報を正確に把握することが重要です。


Generated:

{
    "会社名": "Digital BCG",
    "ポジション名": "Data Scientist",
    "必要な学歴": "Master's degree in a related field",
    "必要な経験年数": "3+ years of work experience in data analytics",
    "必要な言語スキル": "English proficiency (daily conversation level or higher)",
    "必要なテクニカルスキル": "Coding skills in Python, R; Analysis experience using machine learning"
}

=== GENERATION END ===

=== EVALUATION START ===
Status: PASS
Feedback: 
=== EVALUATION END ===

START >>>>>>>>>>  Robust Intelligence - Machine Learning Engineer.pdf

=== GENERATION START ===
Thoughts:

Job Descriptionから必要な情報を抽出するタスクを理解しました。特に、必要な学歴や経験年数、スキルに関する情報を正確に把握することが重要です。フィードバックを受けた場合には、情報の正確性や明確さを向上させるために、より詳細な分析を行う計画です。


Generated:

{
    "会社名": "Robust Intelligence",
    "ポジション名": "Machine Learning Engineer

7

In [11]:
len(reply_results)

7

In [15]:
l0 = []
l1 = []
l2 = []
l3 = []
l4 = []
l5 = []

for r in reply_results:
    jsn = json.loads(r[0])
    print(jsn)
    l0.append(jsn['会社名'])
    l1.append(jsn['ポジション名'])
    l2.append(jsn['必要な学歴'])
    l3.append(jsn['必要な経験年数'])
    l4.append(jsn['必要な言語スキル'])
    l5.append(jsn['必要なテクニカルスキル'])

{'会社名': 'Digital BCG', 'ポジション名': 'Data Scientist', '必要な学歴': "Master's degree in a related field", '必要な経験年数': '3+ years of work experience in data analytics', '必要な言語スキル': 'English proficiency (daily conversation level or higher)', '必要なテクニカルスキル': 'Coding skills in Python, R; Analysis experience using machine learning'}
{'会社名': 'Robust Intelligence', 'ポジション名': 'Machine Learning Engineer', '必要な学歴': 'JDには記載なし', '必要な経験年数': 'At least three years of professional experience with machine learning, ML operations, data science, or a related field', '必要な言語スキル': 'Native level fluency in spoken and written Japanese; Business fluency in spoken and written English', '必要なテクニカルスキル': 'Knowledge and experience of Python; Familiarity with advanced machine learning methods; Experience with tooling and platforms in the ML ecosystem'}
{'会社名': 'Eli Lilly Japan', 'ポジション名': 'Data Scientist – OUS Commercial Analytics', '必要な学歴': 'M.S. or higher in Statistics, Econometrics, Operations Research, Computer Science, Eng

In [18]:
df = pd.DataFrame({'会社名':l0,
                   'ポジション名':l1,
                   '必要な学歴':l2,
                   '必要な経験年数':l3,
                   '必要な言語スキル':l4,
                   '必要なテクニカルスキル':l5})
df

,会社名,ポジション名,必要な学歴,必要な経験年数,必要な言語スキル,必要なテクニカルスキル
0,Digital BCG,Data Scientist,Master's degree in a related field,3+ years of work experience in data analytics,English proficiency (daily conversation level ...,"Coding skills in Python, R; Analysis experienc..."
1,Robust Intelligence,Machine Learning Engineer,JDには記載なし,At least three years of professional experienc...,Native level fluency in spoken and written Jap...,Knowledge and experience of Python; Familiarit...
2,Eli Lilly Japan,Data Scientist – OUS Commercial Analytics,"M.S. or higher in Statistics, Econometrics, Op...",JDには記載なし,Business level proficiency in English; Busines...,Deep and broad knowledge of statistical modeli...
3,OpenAI,Principal Solutions Engineer – Japan,JDには記載なし,7+ years of experience in a Solutions Engineer...,Business level proficiency in English; Busines...,Thorough understanding and knowledge of IT sec...
4,Money Forward,AI/Machine Learning Engineer,JDには記載なし,Experience in practical application of AI/mach...,Basic business level English skills (equivalen...,Experience and understanding of software devel...
5,Amazon Web Services Japan GK,Sr. Generative AI Specialist,Bachelor's degree in computer science or equiv...,5+ years of relevant working experience,JDには記載なし,Experience with machine learning fundamentals;...
6,Dataiku,Senior Data Scientist,JDには記載なし,"8 years + of experience in advanced analytics,...",Japanese - native; highly proficient in spoken...,Over 5 years of experience with Python and SQL...


In [19]:
df.to_csv('../outputs/JD_info.csv', index=False)